In [27]:
import sys
import os
sys.path.insert(0, 'jindal/NER-Bi-LSTM-CNN')


import numpy as np 
# from validation import compute_f1
from keras.models import Model
from keras.layers import TimeDistributed,Conv1D,Dense,Embedding,Input,Dropout,LSTM,Bidirectional,MaxPooling1D,Flatten,concatenate
from prepro import readfile,createBatches,createMatrices,iterate_minibatches,addCharInformatioin,padding
from keras.utils import plot_model,Progbar
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform
from sklearn.metrics import precision_recall_fscore_support as score
import sklearn.metrics

epochs = 50

In [2]:
def tag_dataset(dataset):
    correctLabels = []
    predLabels = []
    b = Progbar(len(dataset))
    for i,data in enumerate(dataset):    
        tokens, casing, labels = data
        tokens = np.asarray([tokens])     
        casing = np.asarray([casing])
#         char = np.asarray([char])
        try:
            pred = model.predict([tokens, casing], verbose=False)[0]   
            pred = pred.argmax(axis=-1) #Predict the classes            
            correctLabels.append(labels)
            predLabels.append(pred)
        except Exception as e:
            continue
        b.update(i)
    return predLabels, correctLabels

def createMatrices(sentences, word2Idx, label2Idx, case2Idx):
    #{'numeric': 0, 'allLower': 1, 'contains_digit': 6, 'PADDING_TOKEN': 7, 'other': 4, 'allUpper': 2, 'mainly_numeric': 5, 'initialUpper': 3}
    unknownIdx = word2Idx['UNKNOWN_TOKEN']
    paddingIdx = word2Idx['PADDING_TOKEN']    
        
    dataset = []
    
    wordCount = 0
    unknownWordCount = 0
    
    for sentence in sentences:
        wordIndices = []    
        caseIndices = []
#         charIndices = []
        labelIndices = []
        
        for word,label in sentence:  
            wordCount += 1
            if word in word2Idx:
                wordIdx = word2Idx[word]
            elif word.lower() in word2Idx:
                wordIdx = word2Idx[word.lower()]                 
            else:
                wordIdx = unknownIdx
                unknownWordCount += 1
#             charIdx = []
#             for x in char:
#                 charIdx.append(char2Idx[x])
            #Get the label and map to int            
            wordIndices.append(wordIdx)
            caseIndices.append(getCasing(word, case2Idx))
#             charIndices.append(charIdx)
            labelIndices.append(label2Idx[label])
           
        dataset.append([wordIndices, caseIndices, labelIndices]) 
        
    return dataset

def getCasing(word, caseLookup):   
    casing = 'other'
    
    numDigits = 0
    for char in word:
        if char.isdigit():
            numDigits += 1
            
    digitFraction = numDigits / float(len(word))
    
    if word.isdigit(): #Is a digit
        casing = 'numeric'
    elif digitFraction > 0.5:
        casing = 'mainly_numeric'
    elif word.islower(): #All lower case
        casing = 'allLower'
    elif word.isupper(): #All upper case
        casing = 'allUpper'
    elif word[0].isupper(): #is a title, initial char upper, then all lower
        casing = 'initialUpper'
    elif numDigits > 0:
        casing = 'contains_digit'
    
   
    return caseLookup[casing]

def iterate_minibatches(dataset,batch_len): 
    start = 0
    for i in batch_len:
        tokens = []
        caseing = []
#         char = []
        labels = []
        data = dataset[start:i]
        start = i
        for dt in data:
            t,c,l = dt
            l = np.expand_dims(l,-1)
            tokens.append(t)
            caseing.append(c)
#             char.append(ch)
            labels.append(l)
        yield np.asarray(labels),np.asarray(tokens),np.asarray(caseing)

In [3]:
def get_sentences(path):
    with open(path,'rb') as f:
        sentences=[]
        sentence=[]
        for line in f:
            splits = [x.decode() for x in line.split()]
            try:
                word = splits[0]
                label = splits[1]
                temp = [word,label]
                sentence.append(temp)
    #             sentence.append(label)
            except Exception as e:
                sentences.append(sentence)
                sentence=[]
    return sentences
# print(sentences[0])
            
        

In [7]:
temp = get_sentences('/srv/Resources/ENP/enp_FR.bnf.train.bio')
testSentences = get_sentences('/srv/Resources/ENP/enp_FR.bnf.test.bio')

trainSentences = temp[:6000]
devSentences = temp[6000:]

print(len(trainSentences))
print(len(devSentences))
print(len(testSentences))


6000
3060
2788


In [5]:
# trainSentences = addCharInformatioin(train_sentences)
# devSentences = addCharInformatioin(dev_sentences)
# testSentences = addCharInformatioin(test_sentences)


In [8]:
print(trainSentences[0])

[['Emmanuel', 'I-PER'], ['DESOLES', 'I-PER'], ['de', 'O'], ['LOU', 'O'], ['Directeur', 'O'], ['politique', 'O'], ['BÊ>ÀCTION', 'O'], ['ET', 'O'], ['ADMINISTRATION', 'O'], ['9&', 'O'], [',', 'O'], ['Rue', 'I-LOC'], ['du', 'I-LOC'], ['Pré-Botté', 'I-LOC'], [',', 'O'], ['aS', 'O'], ['RENNES', 'I-LOC'], ['ABONNEMENTS', 'O'], ['Dép', 'O'], ['.', 'O']]


In [9]:
labelSet = set()
words = {}

for dataset in [trainSentences, devSentences, testSentences]:
    for sentence in dataset:
        for token,label in sentence:
            labelSet.add(label)
            words[token] = True

In [10]:
print(labelSet)

{'I-PER', 'I-ORG', 'I-LOC', 'O'}


In [11]:
# :: Create a mapping for the labels ::
label2Idx = {}
for label in labelSet:
    label2Idx[label] = len(label2Idx)

In [12]:
print(label2Idx)

{'I-PER': 0, 'I-ORG': 1, 'I-LOC': 2, 'O': 3}


In [13]:
# :: Hard coded case lookup ::
case2Idx = {'numeric': 0, 'allLower':1, 'allUpper':2, 'initialUpper':3, 'other':4, 'mainly_numeric':5, 'contains_digit': 6, 'PADDING_TOKEN':7}
caseEmbeddings = np.identity(len(case2Idx), dtype='float32')

In [14]:
string_words = ' '.join(words.keys())
print(string_words)

guérisons ÎO,34 absenter grandc Fournerie- Argonne lui-même étroites Compère-MoreL passait lOSOr Bio-de-Janeiro 19° agréable rejetée Plougras d6 UerMllon AUTOMOBILES guider Luuvel hésitai conclut pru auditions CABANES Assez devien1 \Il àï66 fouet appelés bonhomme airétes Cap autorisé recommenceront laïus Inow-Lodz Jacoby esquiver Chahîert correctionnel mendigot ayons emmenée honme aveuglette grat brillants sergent allée 45 rentre ru66ea 20.483 ILES Cinéma-Uflice SAINTMALO froide Frappa O fournit Refuge Irrévocablement durer 583 vaîentln Debouche certains foice Bot parfait congé Seiae-et-Marne bénéficiaire Gaultier La médecin hmnain vitriers urv déroute Delbay Magenta boisson Constantmople stupitie détendre r>i> inculpés RUR sous-marins p.23(Xi0.1£0(K compUo» n lot attenil cantonnement Jaouen o^tife 502 Discrétion EXPERIMENTE mer Retrouver ordonné épouse <i--9 UNIFORME oi° restés Church Postal collier «oit lézardé ficiel Dien mi-temps QMOnjtral Djézer FontaiaeUitiiu Iparis-France préveu

In [15]:
print(caseEmbeddings)
print(case2Idx)

[[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]
{'numeric': 0, 'initialUpper': 3, 'other': 4, 'contains_digit': 6, 'allUpper': 2, 'mainly_numeric': 5, 'allLower': 1, 'PADDING_TOKEN': 7}


In [16]:
word2Idx={}
wordEmbeddings=[]

# created a file by the name of german_words.txt in /fastText. Containing all the words in our dataset
with open('jindal/NER-Bi-LSTM-CNN/french_word_embeddings.txt','rb') as f:
    for line in f:
        splits = line.split()
        word = splits[0].decode()
#         print(word.decode())
        if len(word2Idx) == 0: #Add padding+unknown
            word2Idx["PADDING_TOKEN"] = len(word2Idx)
            vector = np.zeros(len(splits)-1) #Zero vector vor 'PADDING' word
            wordEmbeddings.append(vector)

            word2Idx["UNKNOWN_TOKEN"] = len(word2Idx)
            vector = np.random.uniform(-0.25, 0.25, len(splits)-1)
            wordEmbeddings.append(vector)
            
        word2Idx[word]=len(word2Idx)
        embedding = np.array([float(num) for num in splits[1:]])
        wordEmbeddings.append(embedding)
wordEmbeddings=np.array(wordEmbeddings)

In [17]:
# print(wordEmbeddings[2])
print(len(wordEmbeddings[0]))
print(word2Idx)

300
{'madeiaoî->': 2, "dis'posait": 3, 'mal': 4, '4955': 5, 'Von': 6, 'Bac': 7, 'attelage': 8, 'trop': 9, 'chc': 10, 'LU': 22441, 'etrf': 12, 'romain': 13, 'aperçu': 14, 'mobilisé': 26331, 'suites': 15, 'fortement': 16, 'sien': 19, 'Lehacre': 20, 'nédifces': 21, 'tué': 22, 'cnmpagnes': 23, 'bu--«in': 24, 'convient': 25, 'Montmédy': 26, 'accéder': 27, 'Ploumimau': 28, 'Bœut': 22443, 'drame': 6011, 'inconnue': 31, '*&': 152, 'carreaux': 33, 'blond': 34, 'Wattelier': 36, 'iraolorisiiques': 37, '1222': 39, 'ûori': 40, 'Matin': 41, 'réélus': 43, 'tinée': 44, 'gnes': 45, 'Garmen': 46, 'trav': 47, 'mangeable': 48, '25j': 49, 'Rannou/': 50, 'SM': 51, 'lavons': 30145, 'Conclusion': 52, 'joints': 275, 'prenutfH.4.5': 296, 'Bernttardt': 56, 'posées': 57, 'mansarde': 58, 'ïïeaux-Arts': 59, 'Rabelais': 60, 'catalogues': 61, 'non': 62, 'Mlawa': 63, 'Telles': 64, 'bavardage': 65, 'sacs': 66, 'Silvestre': 22452, 'Ory': 68, 'HOUlAlMùs': 69, 'brûlaient': 70, '-débauché': 16887, 'pen': 72, 'SAINT': 74, '

In [17]:
print(trainSentences[0]) 

[['Emmanuel', 'I-PER'], ['DESOLES', 'I-PER'], ['de', 'O'], ['LOU', 'O'], ['Directeur', 'O'], ['politique', 'O'], ['BÊ>ÀCTION', 'O'], ['ET', 'O'], ['ADMINISTRATION', 'O'], ['9&', 'O'], [',', 'O'], ['Rue', 'I-LOC'], ['du', 'I-LOC'], ['Pré-Botté', 'I-LOC'], [',', 'O'], ['aS', 'O'], ['RENNES', 'I-LOC'], ['ABONNEMENTS', 'O'], ['Dép', 'O'], ['.', 'O']]


In [18]:
# createMatrices: for every sentence, changes its words, cases,characters, labels to its corresponding id in their embeddings
# padding is used to pad the character indices to a fixed size=52
train_set = createMatrices(trainSentences,word2Idx,  label2Idx, case2Idx,)
dev_set = createMatrices(devSentences,word2Idx, label2Idx, case2Idx)
test_set = createMatrices(testSentences, word2Idx, label2Idx, case2Idx)

In [19]:
# train-set[][0]: corresponds to the ids of the words in the sentence
# train_set[][1]: corresponds to the ids of the cases of the words
# train_set[][2]: contains numpy arrays, one corresponding to every word, each containing the indices of the characters of that word
# the numpy arrays have a fixed size (padding or truncation) to 52
# train_set[][3]: corresponds to the ids of the labels of every word

print(train_set[0])
print(len(train_set[0][0])) # gives the number of words in the sentence
print(len(train_set[0][2]))

[[31846, 21002, 5476, 18253, 15983, 17146, 13432, 3374, 4361, 24599, 8599, 1442, 17229, 516, 8599, 17799, 25282, 16128, 8199, 28964], [3, 2, 1, 2, 3, 1, 2, 2, 2, 6, 4, 3, 1, 3, 4, 4, 2, 2, 3, 4], [0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 3, 3, 2, 3, 3, 3]]
20
20


In [20]:
idx2Label = {v: k for k, v in label2Idx.items()}

train_batch,train_batch_len = createBatches(train_set)
dev_batch,dev_batch_len = createBatches(dev_set)
test_batch,test_batch_len = createBatches(test_set)

In [21]:
words_input = Input(shape=(None,),dtype='int32',name='words_input')
words = Embedding(input_dim=wordEmbeddings.shape[0], output_dim=wordEmbeddings.shape[1],  weights=[wordEmbeddings], trainable=False)(words_input)
casing_input = Input(shape=(None,), dtype='int32', name='casing_input')
casing = Embedding(output_dim=caseEmbeddings.shape[1], input_dim=caseEmbeddings.shape[0], weights=[caseEmbeddings], trainable=False)(casing_input)
# character_input=Input(shape=(None,52,),name='char_input')
# embed_char_out=TimeDistributed(Embedding(len(char2Idx),30,embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5)), name='char_embedding')(character_input)
# dropout= Dropout(0.5)(embed_char_out)
# conv1d_out= TimeDistributed(Conv1D(kernel_size=3, filters=30, padding='same',activation='tanh', strides=1))(dropout)
# maxpool_out=TimeDistributed(MaxPooling1D(52))(conv1d_out)
# char = TimeDistributed(Flatten())(maxpool_out)
# char = Dropout(0.5)(char)
output = concatenate([words, casing,])
output = Bidirectional(LSTM(200, return_sequences=True, dropout=0.50, recurrent_dropout=0.25))(output)
output = TimeDistributed(Dense(len(label2Idx), activation='softmax'))(output)
model = Model(inputs=[words_input, casing_input,], outputs=[output])
model.compile(loss='sparse_categorical_crossentropy', optimizer='nadam')
model.summary()
plot_model(model, to_file='model.png')


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
words_input (InputLayer)        (None, None)         0                                            
__________________________________________________________________________________________________
casing_input (InputLayer)       (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 300)    10112100    words_input[0][0]                
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 8)      64          casing_input[0][0]               
__________________________________________________________________________________________________
concatenat

In [22]:
for epoch in range(epochs):    
    print("Epoch %d/%d"%(epoch,epochs))
    a = Progbar(len(train_batch_len))
    for i,batch in enumerate(iterate_minibatches(train_batch,train_batch_len)):
        labels, tokens, casing = batch       
        model.train_on_batch([tokens, casing], labels)
        a.update(i)
        print(' ')

Epoch 0/50
  1/112 [..............................] - ETA: 4:15 
 
111/112 [============================>.] - ETA: 0s 
Epoch 1/50
  0/112 [..............................] - ETA: 0s 
 
 45/112 [===========>..................] - ETA: 5s 


111/112 [============================>.] - ETA: 0s 
Epoch 2/50
  0/112 [..............................] - ETA: 0s 
 
  2/112 [..............................] - ETA: 5s 
 
 92/112 [=======================>......] - ETA: 1s 


111/112 [============================>.] - ETA: 0s 
Epoch 3/50
  0/112 [..............................] - ETA: 0s 
 
  2/112 [..............................] - ETA: 4s 
 
111/112 [============================>.] - ETA: 0s 
Epoch 4/50
  0/112 [..............................] - ETA: 0s 
 
  2/112 [..............................] - ETA: 4s 
 
 29/112 [======>.......................] - ETA: 6s 


111/112 [============================>.] - ETA: 0s 
Epoch 5/50
  0/112 [..............................] - ETA: 0s 
 
  2/112 [..............................] - ETA: 4s 
 
 76/112 [===================>..........] - ETA: 2s 


111/112 [============================>.] - ETA: 0s 
Epoch 6/50
  0/112 [..............................] - ETA: 0s 
 
  2/112 [..............................] - ETA: 5s 
 
111/112 [============================>.] - ETA: 0s 
Epoch 7/50
  0/112 [..............................] - ETA: 0s 
 
  2/112 [..............................] - ETA: 4s 
 
 13/112 [==>...........................] - ETA: 5s 


...

111/112 [============================>.] - ETA: 0s 
Epoch 45/50
  0/112 [..............................] - ETA: 0s 
 
  2/112 [..............................] - ETA: 4s 
 
 64/112 [================>.............] - ETA: 3s 


111/112 [============================>.] - ETA: 0s 
Epoch 46/50
  0/112 [..............................] - ETA: 0s 
 
  2/112 [..............................] - ETA: 4s 
 
  4/112 [>.............................] - ETA: 4s 
 
111/112 [============================>.] - ETA: 0s 
Epoch 47/50
  0/112 [..............................] - ETA: 0s 
 


  2/112 [..............................] - ETA: 4s 
 
  4/112 [>.............................] - ETA: 4s 
 
111/112 [============================>.] - ETA: 0s 
Epoch 48/50
  1/112 [..............................] - ETA: 7s 
 
 49/112 [============>.................] - ETA: 4s 


111/112 [============================>.] - ETA: 0s 
Epoch 49/50
  0/112 [..............................] - ETA: 0s 
 
  2/112 [..............................] - ETA: 4s 
 
  4/112 [>.............................] - ETA: 4s 
 
 97/112 [========================>.....] - ETA: 1s 


111/112 [============================>.] - ETA: 0s 


In [23]:
import collections
# a = [1,1,1,1,2,2,2,2,3,3,4,5,5]
predLabels, correctLabels = tag_dataset(dev_batch)   
correctLabels = np.concatenate(correctLabels).ravel()
counter=collections.Counter(correctLabels)


3059/3060 [============================>.] - ETA: 0s

In [24]:
print(counter)

Counter({3: 45659, 2: 1256, 0: 1056, 1: 607})


In [25]:
predLabels, correctLabels = tag_dataset(dev_batch)        
predLabels = np.concatenate(predLabels).ravel()
correctLabels = np.concatenate(correctLabels).ravel()
print(idx2Label)

3059/3060 [============================>.] - ETA: 0s{0: 'I-PER', 1: 'I-ORG', 2: 'I-LOC', 3: 'O'}


In [28]:
print(sklearn.metrics.f1_score(correctLabels,predLabels,average='macro' ))
print(sklearn.metrics.f1_score(correctLabels, predLabels, average='micro'))
print(sklearn.metrics.f1_score(correctLabels, predLabels, average='weighted'))
print(sklearn.metrics.f1_score(correctLabels, predLabels, average=None))

0.6789564405222139
0.9591172958952611
0.9597756644101352
[0.69522535 0.41307578 0.62515776 0.98236687]


In [29]:
#   Performance on test dataset       
predLabels, correctLabels = tag_dataset(test_batch)        
predLabels = np.concatenate(predLabels).ravel()
correctLabels = np.concatenate(correctLabels).ravel()

print(sklearn.metrics.f1_score(correctLabels,predLabels,average='macro' ))
print(sklearn.metrics.f1_score(correctLabels, predLabels, average='micro'))
print(sklearn.metrics.f1_score(correctLabels, predLabels, average='weighted'))
print(sklearn.metrics.f1_score(correctLabels, predLabels, average=None))

2785/2788 [============================>.] - ETA: 0s0.7039636857329313
0.9670864398638348
0.967252995681697
[0.72727273 0.38151426 0.72119816 0.9858696 ]


In [30]:
model.save('french_ner_without_char.h5')